# INVERT TOTAL TROPOSPHERIC QGPVP

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import metpy.calc as mpcalc
from metpy.units import units
import sys
import time

from wrf import (getvar, interplevel, to_np, latlon_coords, get_cartopy,
                 cartopy_xlim, cartopy_ylim, extract_times, ALL_TIMES, interpz3d, vinterp)

In [2]:
#Define type of tile and desired pressure level
#fullphys_expdom or nolatentheat_expdom
filetype = "nolatentheat_expdom"                                   #CHANGE
levtitle = "300"

In [3]:
# Open the NetCDF file
ncfile = Dataset("wrfout_"+filetype+".nc", "r")

# Extract the pressure, geopotential height, and wind variables
pressure = getvar(ncfile, "pressure",timeidx=ALL_TIMES)
z = getvar(ncfile, "z", timeidx=ALL_TIMES)
thta = getvar(ncfile, "th",timeidx=ALL_TIMES)
tmpk = getvar(ncfile, "tk",timeidx=ALL_TIMES)
ua = getvar(ncfile, "ua", timeidx=ALL_TIMES, units="kt")
va = getvar(ncfile, "va", timeidx=ALL_TIMES, units="kt")

In [4]:
import pickle
# Getting back the objects:
with open('qgpv_vars_'+levtitle+'_'+filetype+'.pkl', 'rb') as f:
    qgpv, qgpvp, qgpvmean, geop, geopp, geopmean, uamean, vamean, uamean_spec, vamean_spec, SAgeop, SAtmpk, stability = pickle.load(f)

In [5]:
#Define constants
R = 287.05 #J kg^-1 K^-1
kappa=0.2856219 #Rd / cp
cp = 1004.7 #J kg^-1 K^-1
pref = 1000. #hPa
g = 9.80665 #gravity
omega = 7.292e-5 #Earth's rotation rate (s^-1)
a = 2.e7 / np.pi #Radius of the earth (m)
p0=1.e5
alpha=-1./5.255877
gamma = 0.0065 #lapse rate, (K/m)

In [6]:
#Define pressure level array
pb = 1000. #base pressure
pb_half = 975.
pt = 100. #top pressure
pt_half = 125.
p = np.linspace(pb,pt,19) #make array of size 19 with even spacing from base pressure to top pressure
#p_half = np.linspace(pb_half,pt_half,18)
delp = (p[0] - p[1]) * 100. #Convert to hPa

In [7]:
#Manually define phalf based on length of k and pressure values
phalf = np.zeros((len(p)-1))
for i in range(len(p)-1):
    phalf[i] = (p[i+1] + p[i]) / 2.

In [8]:
#Bounds for inversion
lat1 = 30. #Southern boundary
lat2 = 70. #Northern boundary
lon1 = 150. #Western boundary
lon2 = 250. #Eastern boundary
dlat = lat2 - lat1
dlon = lon2 - lon1
dellatstar = (np.pi/180.) * a
dellonstar = (np.pi/180.) * a
sigma1 = dellonstar / dellatstar
sigma2 = dellonstar / delp

In [9]:
#Get times from dataset
times = extract_times(ncfile, None)
count = np.count_nonzero(times)
#Set t, k, i, j indice lengths
tlen = len(times)
klen = len(p)
ilen = np.shape(qgpvp[:,:,:,:])[2] #This is really the j index (north-south index)
jlen = np.shape(qgpvp[:,:,:,:])[3] #This is really the i index (east-west index)

In [10]:
hght = np.zeros((tlen, klen, ilen, jlen))
levels = [1000,950,900,850,800,750,700,650,600,550,500,450,400,350,300,250,200,150,100]
for t in range(tlen):
    hght[t,:,:,:] = vinterp(ncfile,field=z[t,:,:,:],vert_coord='p',interp_levels=levels,extrapolate=True, field_type='z')

In [11]:
lat = getvar(ncfile, "lat")
lon = getvar(ncfile, "lon")

In [12]:
latmax = float(np.max(lat))
latmin = float(np.min(lat))
lonmax = float(np.max(lon))
lonmin = float(np.min(lon))

In [13]:
lat = np.arange(latmin,latmax,((latmax-latmin)/jlen))
lon = np.arange(lonmin,lonmax,((lonmax-lonmin)/ilen))
lenlat = len(lat)
lenlon = len(lon)

In [14]:
#Calculate Coriolis parameter
lenlat = len(lat)
cor = np.zeros((lenlat))
for j in range(lenlat):
    cor[j] = 2. * omega * np.sin(lat[j]*(np.pi / 180.)) # degrees to radians

In [15]:
#Calculate Rossby parameter (beta), variation in Coriolis force with respect to changing latitude
beta = np.zeros((lenlat))
for j in range(lenlat):
    beta[j] = (2. * omega / a) * np.cos(lat[j] * (np.pi / 180.))

In [16]:
#Calculate mean Coriolis Parameter weighted by latitude
fo = 0.
for j in range(lenlat):
    fo = fo + (cor[j] / (dlat+1)) #fo = fo + (cor[j] / (jlen-1))

In [17]:
#Format times to be in standard format
dattimes = []
import pandas as pd
for i in range (0, count):
    dattimes.append(str(pd.Timestamp(times[i])))

## Calculate QGPV coefficients

In [18]:
cosPoint = np.zeros((lenlat))
cosPlus = np.zeros((lenlat)) 
cosPlushalf = np.zeros((lenlat)) 
cosMinus = np.zeros((lenlat)) 
cosMinushalf = np.zeros((lenlat))

for i in range(lenlat):
    cosPoint[i] = np.cos(lat[i] * (np.pi / 180.))
    cosPlus[i] = np.cos((lat[i] + 1.0) * np.pi / 180.)
    cosPlushalf[i] = np.cos((lat[i] + 0.5) * np.pi / 180.)
    cosMinus[i] = np.cos((lat[i] - 1.0) * np.pi / 180.)
    cosMinushalf[i] = np.cos((lat[i] - 0.5) * np.pi / 180.) 

In [19]:
#Coefficients on relative vorticity
A1_rel = np.zeros((lenlat))
A2_rel = np.zeros((lenlat))
A3_rel = np.zeros((lenlat))
A4_rel = np.zeros((lenlat))
A5_rel = np.zeros((lenlat))

for j in range(lenlat):
    A1_rel[j] = 1. / ((cosPoint[j] * dellonstar) ** 2.) # goes on phi(k,i-1,j)
    A2_rel[j] = cosMinushalf[j]/(cosPoint[j]*dellatstar ** 2.) # goes on phi(k,i,j-1)
    A3_rel[j] = (-2. / (cosPoint[j] * dellonstar) ** 2.) - (cosPlushalf[j] / (cosPoint[j] * dellatstar ** 2.)) - (cosMinushalf[j] / (cosPoint[j] * dellatstar ** 2.)) #goes on phi(k,i,j)
    A4_rel[j] = 1. / ((cosPoint[j] * dellonstar) ** 2.) # goes on phi(k,i+1,j) 
    A5_rel[j] = cosPlushalf[j] / (cosPoint[j] * dellatstar ** 2.) # goes on phi(k,i,j+1)
    
    

In [20]:
#Coefficients for QGPV
A1_qgpv = np.zeros((lenlat)) # goes on phi(t,j,i-1,k)
A2_qgpv = np.zeros((lenlat))
A2i_qgpv = np.zeros((lenlat)) # goes on phi(t,j-1,i,k)
A3_qgpv = np.zeros((len(p), lenlat)) # goes on phi(t,j,i,k) has parts in vert, meridional
A3i_qgpv = np.zeros((len(p), lenlat)) # goes on phi(t,j,i,k) has parts in vert, meridional
A4_qgpv = np.zeros((lenlat)) # goes on phi(t,j+1,i,k)
A4i_qgpv = np.zeros((lenlat)) # goes on phi(t,j+1,i,k)
A5_qgpv= np.zeros((lenlat)) # goes on phi(t,j,i+1,k) 
A6_qgpv = np.zeros((len(p))) # goes on phi(t,j,i,k-1) 
A7_qgpv = np.zeros((len(p))) # goes on phi(t,j,i,k+1) 
A8_qgpv = dellonstar ** 2. # goes on q-f 

    
for k in range(2,len(p)-1):
    
    A6_qgpv[k] = (fo * sigma2 ** 2.) / stability[k-1] 
    A7_qgpv[k] = (fo * sigma2 ** 2.) / stability[k]
                   
    for j in range(lenlat):
        A1_qgpv[j] = 1. / (fo * cosPoint[j] ** 2.) 
        A2_qgpv[j] = (sigma1 ** 2.) * cosMinus[j] / (fo * cosPoint[j])
        A2i_qgpv[j] = (sigma1 ** 2.) * cosMinushalf[j] / (fo * cosPoint[j]) 
        A4_qgpv[j] = ((sigma1 ** 2.) * cosPlus[j] / (fo * cosPoint[j]))
        A4i_qgpv[j] = ((sigma1 ** 2.) * cosPlushalf[j] / (fo * cosPoint[j])) 
        A5_qgpv[j] = 1. / (fo * cosPoint[j] ** 2.)
                   
        A3_qgpv[k,j] = -1. * ((2. / (fo * cosPoint[j] ** 2.)) + ((sigma1 ** 2.) / fo) * ((cosPlus[j] / cosPoint[j]) + (cosMinus[j] / cosPoint[j])) + ((4. * fo * sigma2 ** 2.) * ((1. / stability[k-1]) + (1. / stability[k])))) 
        A3i_qgpv[k,j] = (-2. / (fo * cosPoint[j] ** 2.)) + (((sigma1 ** 2.) / fo) * ((-cosPlushalf[j] / cosPoint[j]) - (cosMinushalf[j] / cosPoint[j]))) + ((fo * sigma2 ** 2.) * ((-1. / stability[k-1]) - (1. / stability[k]))) 


In [21]:
#Reshape SAgeop for calculation
SAgeop1 = np.zeros((tlen, klen, ilen, jlen))
for t in range(tlen):
    for k in range(klen):
        SAgeop1[t,k,:,:] = SAgeop[k]

## INVERT THE PERTURBATION UPPER-LEVEL QGPV TO GET THE ASSOCIATED GEOPOTENTIAL

### iterative successive overrelaxation technique is used to attain solutions

### alpha is a parameter to expedite inversion, while the tolerance is set as the change in the solution from the previous to current iteration, summed over all gridpoints. lower tolerance == lower precision but after a certain point you are not gaining any information by enhancing the tolerance

In [22]:
d2r=np.pi/180.
a = 6371.e3
dlon = a*0.5*d2r
dlat = a*0.5*d2r
dp = 50.e2
gamma=.0065
omega = 7.292E-5
f0 = 0.5*(2*omega*(np.sin(d2r*lat[0])+np.sin(d2r*lat[-1])))

In [23]:
#Laplacian coefficient
d2r=np.pi / 180.
a = 6371.e3
dlon = a * 0.5 * d2r
dlat = a * 0.5 * d2r
dp = 50.e2
omega = 7.292E-5
lat_rad=d2r * lat

sigma = (dlon / dlat)
sigma_squared = sigma * sigma

A1 = np.zeros([lenlat])
A2 = np.zeros([lenlat])
A3 = np.zeros([klen,lenlat])
A4 = np.zeros([lenlat])
A5 = np.zeros([lenlat])
A6 = np.zeros([klen])
A7 = np.zeros([klen])     

In [24]:
for k in np.arange(1,klen-1):
    A6[k]=f0/(stability[k-1]*dp**2.)
    A7[k]=f0/(stability[k]*dp**2.)
for k in np.arange(0,klen-1):
    for j in np.arange(1,jlen-1):
        latPlushalf = (lat_rad[j] + lat_rad[j+1]) / 2.
        latMinushalf = (lat_rad[j] + lat_rad[j-1]) / 2.
        
        A1[j]=np.cos(latMinushalf)/(np.cos(lat_rad[j])*(dlat)**2.)
        A2[j]=1./( dlon * np.cos(lat_rad[j]) )**2.
        A3[k,j]=-(A6[k]+A7[k])-(2./(dlon*np.cos(lat_rad[j]))**2. + (np.cos(latMinushalf)+np.cos(latPlushalf))/(np.cos(lat_rad[j])*(dlat)**2.))
        A4[j]=A2[j]
        A5[j]=np.cos(latPlushalf)/(np.cos(lat_rad[j])*(dlat)**2.) 

In [25]:
A1 = dlon * dlon * A1        
A2 = dlon * dlon * A2        
A3 = dlon * dlon * A3        
A4 = dlon * dlon * A4        
A5 = dlon * dlon * A5        
A6 = dlon * dlon * A6        
A7 = dlon * dlon * A7  

In [26]:
#geopper = np.zeros((tlen, klen, ilen, jlen))

#for t in range(tlen):
    #for i in range(ilen):
        #for j in range(jlen):
              #geopper[t,0,i,j] = geopper[t,1,i,j] - ((R*delp)/(975.0*100.))*qgpvp[t,1,i,j]
              #geopper[t,klen-1,i,j] = geopper[t,klen-1,i,j] - ((R*delp)/(975.0*100.))*qgpvp[t,klen-1,i,j]


In [27]:
#set the sides of the domain to the observed geopotential anomaly for
#the levels at which the QGPV anomaly is retained
#for t in range(tlen):
    #for k in range(klen):
        #for i in range(ilen):
            #geopper[t,k,i,0] = geop[t,k,i,0] - geopmean[k,i,0]
            #geopper[t,k,i,jlen-1] = geop[t,k,i,jlen-1] - geopmean[k,i,jlen-1]
        #for j in range(jlen): 
            #geopper[t,k,0,j] = geop[t,k,0,j] - geopmean[k,0,j]
            #geopper[t,k,ilen-1,j] = geop[t,k,ilen-1,j] - geopmean[k,ilen-1,j]
            
            

In [28]:
levtitle = str(input('Enter desired level for inversion (300, 500, 850, or 950): '))
if levtitle == "300":
    krange = np.arange(12,15)
elif levtitle == "500":
    krange = np.arange(8,12)
elif levtitle == "850":
    krange = np.arange(1,5)
elif levtitle == "950":
    krange = np.arange(1,4)

Enter desired level for inversion (300, 500, 850, or 950):  300


In [ ]:
%%time
geopper = np.zeros((tlen, klen, ilen, jlen))
res_list = []
errvar = 10.e-2
for t in np.arange(tlen):
    print("Working on time: ", dattimes[t])
    for iter in np.arange(150):
        error = 0
        for k in (krange):
            for i in np.arange(1,ilen-2):
                for j in np.arange(1,jlen-2):
                    res = A1[j]*geopper[t,k,i,j-1] + A2[j]*geopper[t,k,i-1,j] + A3[k,j]*geopper[t,k,i,j] + A4[j]*geopper[t,k,i+1,j] + A5[j]*geopper[t,k,i,j+1] + A6[k]*geopper[t,k-1,i,j] + A7[k]*geopper[t,k+1,i,j] - qgpvp[t,k,i,j]*dlon*dlon
                    geopper[t,k,i,j] = geopper[t,k,i,j] - 1.9*res/A3[k,j]
                    error = error + abs(res)
                    
            if(k==1): 
                geopper[t,0,:,:] = geopper[t,1,:,:]
            if(k==klen-2): 
                geopper[t,-1,:,:] = geopper[t,-2,:,:]
                        
            
            unit_error = error/((klen-2)*(ilen-2)*(jlen-2))
            res_list.append(unit_error)
            
        if(unit_error < errvar):      # error threshold, average absolute error must be less than specified
            print('amount of error:')
            print('got here: ',error/((ilen-2) * (jlen-2) * (klen-2)), iter)
            break
        #print(geopper[:,27,67])
        #print(iter)
# number of iterations until convergence
print('hmm . . . number of iterations = ', iter)  

In [ ]:
#Plot residual error
plt.plot(res_list)
plt.title("Residual")
plt.show()

In [ ]:
#geopp is observed geopotential anomalies, geopper is calculated geopotential anomalies from inversion
geopanom_obs = geopp
hghtanom_obs = geopp / g

#geopper is from the inversion
#difference between obs and inversion
geop_diff = geopanom_obs - geopper 
hght_diff = geop_diff / g 

#geop_anom_inv is from inversion
hght_anom_inv = geopper / g

In [ ]:
#Calculate geopotential and altitude height by adding back geopmean (phi '' to phi ') and adding back
#the standard atmospheric geopotential height (phi ' to phi)
fullgeopot_inv = (geopper + geopmean) + SAgeop1
fullgeopot_obs = (geopanom_obs + geopmean) + SAgeop1

#Divide by gravity to get altitudinal height (z)
fullhght_obs = fullgeopot_obs / g
fullhght_inv = fullgeopot_inv / g

In [ ]:
#Calculate 3 hourly height tendencies
hghttend_obs = np.zeros((tlen,klen,ilen,jlen))
hghttend_inv = np.zeros((tlen,klen,ilen,jlen))

for t in range(1, tlen-1):
    hghttend_obs[t,:,:,:] = (hght[t+1,:,:,:] - hght[t,:,:,:])
    hghttend_inv[t,:,:,:] = (fullhght_inv[t+1,:,:,:] - fullhght_inv[t,:,:,:])

In [ ]:
hghttend_diff = hghttend_inv - hghttend_obs

In [ ]:
#Save data to file for use in other notebooks
import pickle
with open('inv_qgpv_tot_trop_'+levtitle+'_'+filetype+'.pkl', 'wb') as f:
    pickle.dump([geopper, geopp, hght_anom_inv, hght_diff, hghtanom_obs, fullhght_obs, fullhght_inv, hghttend_obs,hghttend_inv,hghttend_diff], f)

In [ ]:
#Load data from saved file
#import pickle
#with open('inv_qgpv_tot_trop_'+levtitle+'_'+filetype+'.pkl', 'rb') as f:
    #geopper, geopp, hght_anom_inv, hght_diff, hghtanom_obs, fullhght_obs, fullhght_inv, hghttend_obs,hghttend_inv,hghttend_diff = pickle.load(f)

In [ ]:
lats, lons = latlon_coords(z)
dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

In [ ]:
lats = np.asarray(lats)
lons = np.asarray(lons)

In [ ]:
dx = np.asarray(dx)
dy = np.asarray(dy)

In [ ]:
lats_redo = lats
lons_redo = lons

In [ ]:
#Account for cartopy plotting issue when longitude is less than 0.0
for i in range(ilen):
    for j in range(jlen):
        if lons_redo[i,j] < 0.0:
            lons_redo[i,j] = lons_redo[i,j] + 360.
        else:
            lons_redo[i,j] = lons_redo[i,j]

In [ ]:
%%time
#OBSERVED geostrophic wind based on geopotential heights
ug_obs = np.zeros((tlen,klen,ilen,jlen))
vg_obs = np.zeros((tlen,klen,ilen,jlen))
fullgeopot_obs = fullhght_obs * g
for t in np.arange(tlen):
    print("Working on time: ", dattimes[t])
    for k in np.arange(klen):
        for i in np.arange(1, ilen-1):
            for j in np.arange(1, jlen-1):
                ug_obs[t,k,i,j] = -(1. / fo) * ((fullgeopot_obs[t,k,i,j+1] - fullgeopot_obs[t,k,i,j-1]) / (2. * dy[i,j]))
                vg_obs[t,k,i,j] = (1. / fo) * ((fullgeopot_obs[t,k,i+1,j] - fullgeopot_obs[t,k,i-1,j]) / (2. * dx[i,j]))

In [ ]:
%%time
#INVERTED geostrophic wind based on geopotential heights
ug_inv = np.zeros((tlen,klen,ilen,jlen))
vg_inv = np.zeros((tlen,klen,ilen,jlen))
fullgeopot_inv = fullhght_inv * g
for t in np.arange(tlen):
    print("Working on time: ", dattimes[t])
    for k in np.arange(klen):
        for i in np.arange(1, ilen-2):
            for j in np.arange(1, jlen-2):
                ug_inv[t,k,i,j] = -(1. / fo) * ((fullgeopot_inv[t,k,i,j+1] - fullgeopot_inv[t,k,i,j-1]) / (2. * dy[i,j]))
                vg_inv[t,k,i,j] = (1. / fo) * ((fullgeopot_inv[t,k,i+1,j] - fullgeopot_inv[t,k,i-1,j]) / (2. * dx[i,j]))

In [ ]:
if levtitle == "300":
    lev = 14
elif levtitle == "500":
    lev = 10
elif levtitle == "850":
    lev = 3
elif levtitle == "900":
    lev = 2
elif levtitle == "950":
    lev = 1
elif levtitle == "1000":
    lev = 0

In [ ]:
from scipy.ndimage.filters import gaussian_filter

In [ ]:
#PLOTTING FOR GEOPOTENTIAL ANOMALY
for i in range (tlen):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=ncfile)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = hght_anom_inv[i,lev,:,:]
    
    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-100, vmax=100,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-100,-80,-60,-40,-20,20,40,60,80,100]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), to_np(tmp1), levels=levels, transform=crs.PlateCarree(), colors="black")
    plt.clabel(qgcts, fmt='%.1f', inline=True) 

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")


    ax.set_title('WRF '+levtitle+'-hPa Inverted Height Anomaly (m) \n Valid Time: {}'.format(dattimes[i]))
    #($\frac{m}{s^{-2}}$)

    plt.savefig('plots/inv_tot_trop/'+filetype+'/full_qgpv/invheightanom/invheightanom_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR HEIGHT DIFFERENCE
for i in range (tlen):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=ncfile)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = hght_diff[i,lev,:,:]

    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-100, vmax=100,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-100,-80,-60,-40,-20,20,40,60,80,100]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), to_np(tmp1), transform=crs.PlateCarree(), colors="black")
    plt.clabel(qgcts, fmt='%.1f', inline=True) 

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")


    ax.set_title('WRF '+levtitle+'-hPa Height Difference (m), Observation -  Inversion \n Valid Time: {}'.format(dattimes[i]))
    
    plt.savefig('plots/inv_tot_trop/'+filetype+'/full_qgpv/heightdiff/heightdiff_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR OBS HEIGHT ANOMALY
for i in range (0,count):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=ncfile)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = np.squeeze(hghtanom_obs[i,lev,:,:])

    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-250., vmax=250.,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-260,-240,-220,-200,-180,-160,-140,-120,-100,-80,-60,-40,-20,20,40,60,80,100,120,140,160,180,200,220,240,260]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), to_np(tmp1), transform=crs.PlateCarree(), colors="black")
    plt.clabel(qgcts, fmt='%.1f', inline=True) 

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    ax.set_title('WRF '+levtitle+'-hPa Observed Height Anomaly (m) \n Valid Time: {}'.format(dattimes[i]))
    
    plt.savefig('plots/inv_tot_trop/'+filetype+'/full_qgpv/obsheightanom/obsheightanom_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR INV HEIGHT
for i in range (0,count):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=ncfile)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = np.squeeze(fullhght_inv[i,lev,:,:])

    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='coolwarm', vmin=8600., vmax=9600.,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    if lev == 1:
        levels = np.arange(0,760,20)
    if lev == 14:
        levels = np.arange(8000,11000,100)
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors='black')
    plt.clabel(qgcts, fmt='%.1f', inline=True) 
    
    tmp2 = np.squeeze(ug_inv[i,lev,:,:])
    tmp3 = np.squeeze(vg_inv[i,lev,:,:])
    
    Q = plt.barbs(to_np(lons[::15,::15]),to_np(lats[::15,::15]), tmp2[::15,::15], tmp3[::15,::15], 
                transform=crs.PlateCarree(), length = 5)

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    ax.set_title('WRF '+levtitle+'-hPa Inverted Height (m) and Geostrophic Winds (kts) \n Valid Time: {}'.format(dattimes[i]))
    
    plt.savefig('plots/inv_tot_trop/'+filetype+'/full_qgpv/invheight/invheight_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR OBS HEIGHT
for i in range (0,count):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=ncfile)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = np.squeeze(hght[i,lev,:,:])

    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='coolwarm', vmin=8600, vmax=9600.,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    if lev == 1:
        levels = np.arange(0,800,20)
    if lev == 14:
        levels = np.arange(8000,11000,100)
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors='black')
    plt.clabel(qgcts, fmt='%.1f', inline=True) 
    
    # Add the wind barbs, only plotting every 15th data point.
    plt.barbs(to_np(lons[::15,::15]), to_np(lats[::15,::15]),
          to_np(ug_obs[i,lev, ::15, ::15]), to_np(vg_obs[i,lev, ::15, ::15]),
          transform=crs.PlateCarree(), length=5)
    
    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    ax.set_title('WRF '+levtitle+'-hPa Observed Height (m) and Geostrophic Winds (kts) \n Valid Time: {}'.format(dattimes[i]))
    
    plt.savefig('plots/inv_tot_trop/'+filetype+'/full_qgpv/obsheight/obsheight_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR INV HEIGHT TENDENCY
for i in range (1,count-1):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=ncfile)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = np.squeeze(hghttend_inv[i,lev,:,:])

    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-150., vmax=150.,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    levels = [-100,-80,-60,-40,-20,20,40,60,80,100]
    #FOR PROPER, PAPER-READY PLOTS
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors="black")
    plt.clabel(qgcts, fmt='%.1f', inline=True) 

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    ax.set_title('WRF '+levtitle+'-hPa Inverted Height Tendency (m / 3 hr) \n Valid Time: {}'.format(dattimes[i]+' to '+dattimes[i+1]))
    
    plt.savefig('plots/inv_tot_trop/'+filetype+'/full_qgpv/invheighttend/invheighttend_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR OBS HEIGHT TENDENCY
for i in range (1,count-1):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=ncfile)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = np.squeeze(hghttend_obs[i,lev,:,:])

    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-150., vmax=150.,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    if lev == 1:
        levels = [-10,-8,-6,-4,-2,2,4,6,8,10]
    if lev == 14:
        levels = [-100,-80,-60,-40,-20,20,40,60,80,100]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors="black")
    plt.clabel(qgcts, fmt='%.1f', inline=True)

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    ax.set_title('WRF '+levtitle+'-hPa Observed Height Tendency (m / 3 hr) \n Valid Time: {}'.format(dattimes[i]+' to '+dattimes[i+1]))
    
    plt.savefig('plots/inv_tot_trop/'+filetype+'/full_qgpv/obsheighttend/obsheighttend_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR HEIGHT TENDENCY DIFFERENCE (INVERSION - OBSERVATION)
for i in range (1,count-1):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=ncfile)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = np.squeeze(hghttend_diff[i,lev,:,:])

    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-150., vmax=150.,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-100,-80,-60,-40,-20,20,40,60,80,100]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors="black")
    plt.clabel(qgcts, fmt='%.1f', inline=True)

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    ax.set_title('WRF '+levtitle+'-hPa Height Tendency Difference (m / 3 hr) \n Inversion - Observation \n Valid Time: {}'.format(dattimes[i]+' to '+dattimes[i+1]))
    
    plt.savefig('plots/inv_tot_trop/'+filetype+'/full_qgpv/heighttenddiff/heighttenddiff_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#Melissa's inversion calculation

#import time
#start = time.time()

#for t in range(tlen):
    #print("Working on time: ", dattimes[t])
    #for m in range(10): #Fast inversion: m = 10, Slow inversion: m = 5000
        #check = 0
        #for k in range(1, klen-1):
            #print("I am here: ", k)
            #for i in range(1, ilen-1):
                #for j in range(1, jlen-1):
                    #A3i_qgpv[A3i_qgpv==0] = np.nan
                    #geoppertemp = ((qgpvp[t,k,i,j])*A8_qgpv-A1_qgpv[j]*geopper[t,k,i-1,j]-A2i_qgpv[j]*geopper[t,k,i,j-1]-A4i_qgpv[j]*geopper[t,k,i,j+1]-A5_qgpv[j]*geopper[t,k,i+1,j]-A6_qgpv[k]*geopper[t,k-1,i,j]-A7_qgpv[k]*geopper[t,k+1,i,j])/A3i_qgpv[k,j]
                    #res = geopper[t,k,:,:] - geoppertemp
                    #geopper[t,k,:,:] = geopper[t,k,:,:] - (alpha * res)
                    #error = error + abs(res)
                    #print(error)
                    
                    # incorporate upper/lower level boundary theta into solution 
                    #if (k == 1):
                        #geopper[t,0,:,:] = geopper[t,1,:,:] - ((Rconst*delp)/(phalf[0]*100.))*qgpvp[t,0,:,:]                  
                    #elif (k == 30):
                        #geopper[t,31,:,:] = geopper[t,30,:,:] + ((Rconst*delp)/(phalf[-1]*100.))*qgpvp[t,31,:,:]
    #if (check < tol):
        #print("Woohoo! Convergence!")
        #break
        
    #elif (m < 5000):
        #print("m= ", m)
        #print("check= ", check)
        
    #elif (m == 5000):
        #print("No convergence reached...try again")
        #break
        
    #print("I am here: ", m)

#print ("Clock time to run= ",time.time()- start," seconds")                 
                    